# asyncio

Определения:

- Concurrency (конкурентность) — две или более задачи могут запускаться, выполняться и завершаться в перекрывающиеся периоды времени (наиболее общее понятие)
- Parallel execution (параллелизм) — исполнение нескольких задач одновременно, например, при помощи многоядерного процессора
- Multithreading (многопоточность) — один из способов реализации конкурентности путем выделения абстракции "рабочего потока" (возможна и на многоядерных, и на одноядерных процессорах)
- Asynchrony (асинхронность) — возникновение событий, которые происходят одновременно с выполнением программы, без блокировки программы для ожидания результатов

#### Важно понимать разницу между параллелизмом и конкурентностью!

### Выделим 2 типа операций:
    
* CPU-bound — нагружают вычислительные мощности текущего устройства
* IO-bound — связаны с длительным ожиданием другого устройства, например, сетевой карты или диска

Пример операции с длительным ожиданием:

In [ ]:
with open('large_file.txt') as f:
    # blocks until OS reads all the data
    data = f.read()

Чтение из большого файла может занимать несколько секунд, но эта операция блокирующая, хотя процессор мог бы в это время совершать другую полезную работу. Процессор "простаивает" пока не произойдет чтение всего файла.

Еще один пример (мы ждем, пока не получим ответ с сайта, хотя могли бы в это время выполнять полезную работу и продолжить исполнение этого кода как только получили бы ответ):

In [ ]:
import requests

# blocks until site returns response
response = requests.get('http://very.slow.site')

### Асинхронные операции

Асинхронные операции чтения/записи позволяют программе продолжать выполнение, не дожидаясь результата их исполнения.

С помощью механизма callback'ов можно реализовать требуемую логику.

### Пример:

In [ ]:
# функция, отвечающая за обработку ответа
def handle_response(response):
    print('\n{:.70}...'.format(response.body))

# создание объекта для общения с сетью
http_client = AsyncHTTPClient()

# неблокирующий вызов функции!
# после вызова функции fetch будет выполняться следующий за этой строчкой код без ожидания получения ответа
# ответ с сайта будет обработан функцией handle_response (так называемым callback'ом)
http_client.fetch('http://yandex.ru', callback=handle_response)

### Ад обратных вызовов (callback hell)

Проблема данного подхода заключается в том, что внутри одной callback функции может быть вызвана другая и т.д. Такой код становится трудно читаем.

Для этого были придуманы корутины (coroutines)

### Корутины

(функция/генератор, которая умеет взаимодействовать с event loop'ом)

https://docs.python.org/3/library/asyncio-task.html

### Немного экскурса в историю

Python 2.2 (генераторы, ключевое слово - yield):

In [ ]:
def lazy_range(up_to):
    index = 0
    while index < up_to:
        yield index
        index += 1

Python 3.3 добавляется важный синтаксический сахар **yield from**:

In [6]:
def g(x):
    yield from range(x, 0, -1)
    yield from range(x)

list(g(5))

[5, 4, 3, 2, 1, 0, 1, 2, 3, 4]

В Python 3.4 появляется фреймворк asyncio:

In [2]:
import asyncio

И становится возможным написать:

In [11]:
# корутина
@asyncio.coroutine
def countdown(label, n):
    while n > 0:
        print('{}: {}'.format(label, n))
        yield from asyncio.sleep(1)
        n -= 1

# цикл событий (подробнее об этом чуть позже)
loop = asyncio.get_event_loop()
tasks = [
    countdown('A', 2),
    countdown('B', 3)
]
loop.run_until_complete(asyncio.wait(tasks))

A: 2
B: 3
A: 1
B: 2
B: 1


({<Task finished coro=<countdown() done, defined at <ipython-input-11-22bc4cdd050e>:2> result=None>,
  <Task finished coro=<countdown() done, defined at <ipython-input-11-22bc4cdd050e>:2> result=None>},
 set())

Синтаксически корутина очень сильно напоминает генератор, хотя имеет совершенно другой смысл.

Для избежания путаницы, в **Python 3.5** вводят ключевые слова **async/await**, окончательно скрыв тот факт, что корутина - это всё тот же генератор.

### Начиная с Python 3.5 возможно написать:

In [9]:
# Корутина
async def compute(a, b):
    print("Compute...")
    await asyncio.sleep(1.0)
    return a + b

Разберемся с asyncio. Для начала выделим понятия, которыми оперирует asyncio:

* **цикл событий** (event loop) по большей части всего лишь управляет выполнением различных задач: регистрирует поступление и запускает в подходящий момент
* **корутины** — специальные функции, похожие на генераторы python, от которых ожидают (await), что они будут отдавать управление обратно в цикл событий. Необходимо, чтобы они были запущены именно через цикл событий
* **футуры** — объекты, в которых хранится текущий результат выполнения какой-либо задачи. Это может быть информация о том, что задача ещё не обработана или уже полученный результат; а может быть вообще исключение

C помощью синтаксиса **await** мы определяем места, где можно переключиться на другие ожидающие выполнения задачи.

Посмотрим на то, как это работает:

In [3]:
async def foo():
    print('Running in foo')
    await asyncio.sleep(0) # здесь возможно переключение на другую задачу
    print('Explicit context switch to foo again')

async def bar():
    print('Explicit context to bar')
    await asyncio.sleep(0) # здесь также возможно переключение на другую задачу
    print('Implicit context switch back to bar')

ioloop = asyncio.get_event_loop()  # получение event loop'а главного потока
tasks = [ioloop.create_task(foo()), ioloop.create_task(bar())] 
wait_tasks = asyncio.wait(tasks)
ioloop.run_until_complete(wait_tasks)

Running in foo
Explicit context to bar
Explicit context switch to foo again
Implicit context switch back to bar


({<Task finished coro=<foo() done, defined at <ipython-input-3-5c702e99f3f9>:1> result=None>,
  <Task finished coro=<bar() done, defined at <ipython-input-3-5c702e99f3f9>:6> result=None>},
 set())

Корутины содержат операторы yield, с помощью которых мы определяем места, где можно переключиться на другие ожидающие выполнения задачи.

За переключение контекста в asyncio отвечает yield, который передаёт управление обратно в event loop, а тот в свою очередь — к другой корутине.

Используя **await** в какой-либо корутине, мы, таким образом, объявляем, что корутина может отдавать управление обратно в event loop, который, в свою очередь, запустит какую-либо следующую задачу: bar. В bar произойдёт тоже самое: на await asyncio.sleep управление будет передано обратно в цикл событий, который в нужное время вернётся к выполнению foo.

Еще один пример (с получением результата):

In [5]:
async def compute(a, b):
    print('Compute...')
    await asyncio.sleep(1.0)
    return a + b

async def print_sum(a, b):
    result = await compute(a, b)
    print('{} + {} = {}'.format(a, b, result))

loop = asyncio.get_event_loop()
loop.run_until_complete(print_sum(1, 2))

Compute...
1 + 2 = 3


<Вставить картинку>

Еще один пример на создание и управление тасками:

In [ ]:
import asyncio

async def say(what, when):
    await asyncio.sleep(when)
    print(what)

async def stop_after(loop, when):
    await asyncio.sleep(when)
    loop.stop()


loop = asyncio.get_event_loop()

loop.create_task(say('first hello', 2))
loop.create_task(say('second hello', 1))
loop.create_task(say('third hello', 4))
loop.create_task(stop_after(loop, 3))

loop.run_forever()

Начиная с Python 3.7 синтаксис упростился еще сильнее:

In [ ]:
import asyncio

async def main():
    print('Hello ...')
    await asyncio.sleep(1)
    print('... World!')

# Python 3.7+
asyncio.run(main())

### Упражнение №1

Что будет напечатано и почему?

In [4]:
async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({i})...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")

async def main():
    await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

Task B: Compute factorial(2)...
Task C: Compute factorial(2)...
Task A: Compute factorial(2)...
Task B: Compute factorial(3)...
Task C: Compute factorial(3)...
Task A: factorial(2) = 2
Task B: factorial(3) = 6
Task C: Compute factorial(4)...
Task C: factorial(4) = 24


### Waiting & timeouts

Пример на выставление timeout:

In [7]:
async def eternity():
    # Sleep for one hour
    await asyncio.sleep(3600)
    print('yay!')

async def main():
    # Wait for at most 1 second
    try:
        await asyncio.wait_for(eternity(), timeout=1.0)
    except asyncio.TimeoutError:
        print('timeout!')

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

timeout!


In [8]:
async def factorial(number):
    f = 1
    for i in range(2, number + 1):
        await asyncio.sleep(1)
        f *= i
    return number, f

async def main():
    for fut in asyncio.as_completed([factorial(4), factorial(3),
                                     factorial(5), factorial(2)]):
        number, result = await fut
        print(f"Factorial({number}) = {result}")  # печатается каждый раз как только будет выполнена какая-либо таска
        
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

Factorial(2) = 2
Factorial(3) = 6
Factorial(4) = 24
Factorial(5) = 120


## async with
Асинхронный контекстный менеджер - это контекстный менджер, который умеет приостанавливать выполнение в методах входа и выхода: \__aenter\__(), \__aexit\__()

In [ ]:
lock = asyncio.Lock()

# ... later
await lock.acquire()
try:
    # access shared state
finally:
    lock.release()

In [ ]:
lock = asyncio.Lock()

# ... later
async with lock:
    # access shared state

### aiohttp

Рядом с asyncio создано огромное количество асинхронных модулей для решения всевозможных задач. **aiohttp** - лишь одна из них. Это асинхронный HTTP Клиент/Сервер

В следующем примере получаем содержимое страницы google.com:

In [ ]:
import aiohttp

async with aiohttp.ClientSession() as session:
    async with session.get('http://google.com') as resp:
        text = await resp.text()
        print('{:.70}...'.format(text))

Реализация простого сервера:

In [11]:
from aiohttp import web

async def handle(request):
    name = request.match_info.get('name', 'Anonymous')
    text = 'Hello, ' + name
    # ...
    # здесь идет некоторая дополнительная логика с async/await
    #
    return web.Response(text=text)

app = web.Application()
app.add_routes([web.get('/', handle),
                web.get('/{name}', handle)])

web.run_app(app)

======== Running on http://0.0.0.0:8080 ========
(Press CTRL+C to quit)


### Задача №1

Узнать свой IP адрес. Есть куча сервисов, которые позволяют узнать ваш ip. Но на момент запуска программы вы не знаете какой из сервисов доступен. Вместо того, чтобы опрашивать каждый из этих сервисов последовательно, можно запустить все запросы конкурентно и выбрать первый успешный.

Потребуется **asyncio.wait()** и параметр **return_when**

In [ ]:
from collections import namedtuple
import time
import asyncio
from concurrent.futures import FIRST_COMPLETED
import aiohttp

Service = namedtuple('Service', ('name', 'url', 'ip_attr'))

SERVICES = (
    Service('ipify', 'https://api.ipify.org?format=json', 'ip'),
    Service('ip-api', 'http://ip-api.com/json', 'query')
)

async def fetch_ip(service):
    # получение ip


async def asynchronous():
    # TODO:
    # создание футур для сервисов
    # используйте FIRST_COMPLETED

ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())

Для правильной реализации немного теории.

Возможные состояния футур:
- ожидание (pending)
- выполнение (running)
- выполнено (done)
- отменено (cancelled)

Когда футура находится в состояние **done**, у неё можно получить результат выполнения. В состояниях **pending** и **running** такая операция приведёт к исключению **InvalidStateError**, а в случае **canelled** будет **CancelledError**, и наконец, если исключение произошло в самой корутине, оно будет сгенерировано снова (также, как это сделано при вызове exception).

Узнать состояние футуры с помощью методов **done**, **cancelled** или **running**, но не забывайте, что в случае **done** вызов **result** может вернуть как ожидаемый результат, так и исключение, которое возникло в процессе работы. 

Для отмены выполнения футуры есть метод **cancel** (он то нам и требуется для корректного завершения работы)

Теперь мы изучили достаточно для того, чтобы написать простого чат бота, который будет делать что-то полезное.

### Задача №2

Напишите телеграм бота, который будет на сообщение присылать соответствующее изображение

* установить aiogram 1.4 - асинхронная обертка над api телеграмма
* поговорить с @FatherBot, создать бота и запомнить выданный токен
* В рф нужно использовать впн или прокси (в сети есть огромное количество списков адресов)
* разобраться с примером эхо бота ниже
* написать требуемый функционал (картинки можно запрашивать через поиск яндекса или гугла, существуют готовые api, можно написать и самостоятельно)

In [ ]:
from aiogram import Bot, types
from aiogram.dispatcher import Dispatcher
from aiogram.utils import executor

PROXY_URL = 'socks5://xxx.xxx.xxx.xxx' # вставить здесь подходящий ip

secret_token = 'XXX'  # строка вида: 123456789:ABCDEFGHJABCDEFGHJABCDEFGHJABCDEFGHJ

bot = Bot(token=secret_token, proxy=PROXY_URL)
dp = Dispatcher(bot)


@dp.message_handler(commands=['start', 'help'])
async def send_welcome(message: types.Message):
    await message.reply("Hi!\nI'm EchoBot!\nPowered by aiogram.")


@dp.message_handler()
async def echo(message: types.Message):
    await message.reply(message.text)


if __name__ == '__main__':
    executor.start_polling(dp)